In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime

station_dataframes = []

url = "https://www.meteoclimatic.net/index/mapinfo/ESCAT?d=20231021&mp=0"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

def dms_to_decimal(dms_str):
    matches = re.match(r"(\d+)º (\d+)' (\d+)'' ([NSEW])", dms_str)
    if not matches:
        return None

    degrees, minutes, seconds, direction = matches.groups()
    decimal = float(degrees) + float(minutes)/60 + float(seconds)/3600
    
    if direction in ['S', 'W']:
        decimal = -decimal

    return decimal

station_links = [link for link in soup.select("a") if link.get('href') and link['href'].startswith('/perfil/')]

print()
print(f'{len(station_links)} stations were found!')
print()

for link in station_links:
   
    station_name = link.text.strip()  
    print(f'################### PROCESSING {station_name} ###################')
    print()
    station_url = "https://www.meteoclimatic.net" + link['href'].strip()  
    response2 = requests.get(station_url)
    station_soup = BeautifulSoup(response2.content, 'html.parser')
    
    # latitude & longitude
    lat_lon_elements = station_soup.find_all(string=lambda text: "º" in text and "''" in text)
    if lat_lon_elements and len(lat_lon_elements[0].split("\xa0\xa0\xa0\xa0")) >= 2:
        lat_str, lon_str = lat_lon_elements[0].split("\xa0\xa0\xa0\xa0")[0:2]
        latitude_decimal = dms_to_decimal(lat_str)
        longitude_decimal = dms_to_decimal(lon_str)
    #print(f'Latitude = {latitude_decimal}')
    #print(f'Longitude = {longitude_decimal}')
    
    # altitude
    altitude_pattern = r"\d+º.*\d+'' [NESW]\s+-\s+(\d+)"
    match_refined = re.search(altitude_pattern, station_soup.get_text())
    if match_refined:
        altitude_value = match_refined.group(1)
    else:
        altitude_value= "Altitude not found"
    #print()
    #print(f'Altitude = {altitude_value}')
    #print()
    
    current_weather_container = station_soup.find(string=lambda text: "EL TIEMPO ACTUAL" in text if text else False)
    if not current_weather_container:
        print("Current weather text not found")

    weather_data_table = current_weather_container.find_parent('table')
    if not weather_data_table:
        print("Weather data table not found")
    
    parent_container = weather_data_table.find_parent()
    parent_container_content = [str(child)[:500] for child in parent_container.children]
    weather_data_table_v2 = parent_container.find('table', cellpadding="2", cellspacing="1")
    weather_data_table_v2.prettify()[:500] if weather_data_table_v2 else "Table not found"
    rows_v2 = weather_data_table_v2.find_all('tr')
    headers = [header.get_text(strip=True) for header in rows_v2[0].find_all('td') if 'class' in header.attrs and 'titolet' in header.attrs['class']]
    headers.insert(3, 'Dirección Viento')
    
    data = []
    for row in rows_v2[1:]:
        row_data = [cell.get_text(strip=True) for cell in row.find_all('td') if 'class' not in cell.attrs or 'separador' not in cell.attrs['class']]
        if row_data:  
            data.append(row_data)
    
    structured_data = {
        "Temperatura": {},
        "Humedad": {},
        "Viento": {},
        "Presión": {},
        "Precip.": {},
        "Días Sequía": {}
    }
    
    def extract_wind_data(wind_data):
        parts = wind_data.split(' ')
        if len(parts) > 1:
            direction, speed = parts[0], parts[1]
        else:
            direction, speed = "", wind_data
        return direction, speed

    wind_direction, wind_speed = extract_wind_data(data[1][2])
    
    if len(data) > 1 and len(data[1]) > 5:
        structured_data["Temperatura"]["Actual"] = data[1][0]
        structured_data["Humedad"]["Actual"] = data[1][1]
        structured_data["Viento"]["Actual"] = {"Dirección": extract_wind_data(data[1][2])[0], "Velocidad": extract_wind_data(data[1][2])[1]}
        structured_data["Presión"]["Actual"] = data[1][3]
        structured_data["Precip."]["Actual"] = data[1][4]
        structured_data["Días Sequía"]["Actual"] = data[1][5]
    else:
        continue
        print(f"Not enough data for station: {station_name}")
        
    for time_period, row in zip(["Hoy", "Mes", "Año"], data[3:]):
        if len(row) > 1:
            structured_data["Temperatura"][time_period] = {"Máx.": row[1], "Mín.": row[2]} if len(row) > 2 else {"Máx.": row[1], "Mín.": None}
        else:
            structured_data["Temperatura"][time_period] = {"Máx.": None, "Mín.": None}
            print(f"Not enough temperature data for {time_period} in station: {station_name}")

        if len(row) > 4:
            structured_data["Humedad"][time_period] = {"Máx.": row[4], "Mín.": row[5]} if len(row) > 5 else {"Máx.": row[4], "Mín.": None}
        else:
            structured_data["Humedad"][time_period] = {"Máx.": None, "Mín.": None}
            print(f"Not enough humidity data for {time_period} in station: {station_name}")

        if len(row) > 7:
            structured_data["Viento"][time_period] = {"Máx.": row[7]}
        else:
            structured_data["Viento"][time_period] = {"Máx.": None}
            print(f"Not enough wind data for {time_period} in station: {station_name}")

        if len(row) > 9:
            structured_data["Presión"][time_period] = {"Máx.": row[9], "Mín.": row[10]} if len(row) > 10 else {"Máx.": row[9], "Mín.": None}
        else:
            structured_data["Presión"][time_period] = {"Máx.": None, "Mín.": None}
            print(f"Not enough pressure data for {time_period} in station: {station_name}")

        if len(row) > 12:
            structured_data["Precip."][time_period] = {"Total": row[12]}
        else:
            structured_data["Precip."][time_period] = {"Total": None}
            print(f"Not enough precipitation data for {time_period} in station: {station_name}")
    
    station_df = pd.DataFrame({
        'Station Name': [station_name],
        'Latitude': [latitude_decimal],
        'Longitude': [longitude_decimal],
        'Altitude': [altitude_value],
        'Temp_max': [structured_data["Temperatura"]["Hoy"]["Máx."]],
        'Temp_min': [structured_data["Temperatura"]["Hoy"]["Mín."]],
        'Wind_max': [structured_data["Viento"]["Hoy"]["Máx."]],
        'Precip': [structured_data["Precip."]["Hoy"]["Total"]],
    })

    numerical_columns = ['Latitude', 'Longitude', 'Altitude', 'Temp_max', 'Temp_min', 'Wind_max', 'Precip']
    for column in numerical_columns:
        station_df[column] = pd.to_numeric(station_df[column], errors='coerce')
    
    print(station_df.to_string(index=False))
    print()
    
    station_dataframes.append(station_df)

final_df = pd.concat(station_dataframes, ignore_index=True)
final_df = final_df.dropna() 

today = datetime.date.today()
date = today.strftime("%d%m%Y")
final_df.to_csv(f'meteoclimatic_{date}.csv', index=False)


453 stations were found!

################### PROCESSING Cercedilla (Madrid) ###################

       Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Cercedilla (Madrid) 40.735278  -4.073056      1243      12.6       6.6        26     1.6

################### PROCESSING Aiguafreda ###################

Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
  Aiguafreda 41.768889   2.250278       420      21.2       5.4        20     0.0

################### PROCESSING Aiguafreda Aj. ###################

  Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Aiguafreda Aj. 41.770833   2.250556       427      20.5       5.6        36     NaN

################### PROCESSING Alella-Mirador ###################

  Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Alella-Mirador   41.4925   2.291389       290      18.9      12.9        39     0.0

################### PR

        Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Barcelona - Rectoret 41.425556   2.099722       320      17.9       9.7      1007     0.0

################### PROCESSING Barcelona - Sant Andreu ###################

################### PROCESSING Barcelona - Sant Andreu ###################

           Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Barcelona - Sant Andreu 41.429722     2.1925        50      20.2      13.1        48     NaN

################### PROCESSING Barcelona - Sant Gervasi ###################

            Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Barcelona - Sant Gervasi   41.4125   2.139167       160      19.3      11.8        35     0.0

################### PROCESSING Barcelona - Sant Martí ###################

          Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Barcelona - Sant Martí     41.41   2.190833    

 Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Castelldefels   41.2675      1.935        62      18.7      13.4        29     0.0

################### PROCESSING Castellgali ###################

Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
 Castellgali       NaN   1.837778       266      21.7       7.1        23     0.0

################### PROCESSING Castellterçol ###################

 Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Castellterçol 41.752222   2.128333       729      17.4       2.7        15     NaN

################### PROCESSING Centelles ###################

Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
   Centelles 41.775278   2.241389       500      19.1       5.3        26     NaN

################### PROCESSING Centelles - Oest ###################

    Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_m

        Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
L'Ametlla del Vallès 41.699167   2.243611       647      17.8       7.8        34     0.0

################### PROCESSING L'Espunyola ###################

Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
 L'Espunyola 42.052222   1.772778       802      13.0       2.9        34     0.2

################### PROCESSING La Batllòria - Vallès Ori ###################

             Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
La Batllòria - Vallès Ori 41.718056   2.547778        97      21.8       7.0        40     NaN

################### PROCESSING La Beguda Alta ###################

  Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
La Beguda Alta 41.497222   1.836111       285      19.6       8.6        27     0.0

################### PROCESSING La Floresta (St Cugat) ###################

          

################### PROCESSING Montornès del Vallès ###################

        Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Montornès del Vallès 41.541667   2.266389       100      20.4      11.5        13     NaN

################### PROCESSING Montseny - Can Cervera ###################

          Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Montseny - Can Cervera     41.77   2.390833       710      18.1       7.6        43     0.0

################### PROCESSING Montseny - Les Arrels ###################

         Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Montseny - Les Arrels 41.760556   2.393889       435      20.4       8.8        27     NaN

################### PROCESSING Olesa Montserrat-Eixample ###################

             Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Olesa Montserrat-Eixample 41.540556   1.895556       110

...

             Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Montessori Palau Figueres 42.266111   2.998611        15      21.9       6.1        48     NaN

################### PROCESSING Montfullà ###################

Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
   Montfullà 41.963889   2.758889       180      22.0       8.8        23     0.0

################### PROCESSING Olot - Costa del Gegant ###################

           Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Olot - Costa del Gegant 42.165833   2.499722       509      17.1       2.9        66     0.0

################### PROCESSING Olot - Pla de Dalt ###################

      Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Olot - Pla de Dalt 42.185556   2.469444       452      17.3       2.7        92     NaN

################### PROCESSING Palafrugell ###################

#######

  Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Verges (Ter7). 42.056944   3.049167        15      23.0       6.1        52     0.0

################### PROCESSING Verges Ponent ###################

 Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Verges Ponent 42.060278   3.045833        27      23.1       5.9        66     NaN

################### PROCESSING Vidreres ###################

Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
    Vidreres       NaN   2.779444        98      22.1       5.4        35     0.0

################### PROCESSING Vilablareix ###################

Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
 Vilablareix 41.952778   2.793611       107      22.1       6.9        55     0.0

################### PROCESSING Alpicat ###################

Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
 

    Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Refugi d'Amitges 42.596667   0.984722      2380       3.6      -4.1        60    10.6

################### PROCESSING Refugi Josep Mª Blanc ###################

################### PROCESSING Rialp ###################

Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
       Rialp 42.444167   1.135556       725       8.5       5.2        18     0.0

################### PROCESSING Rubió ###################

Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
       Rubió 42.376389   1.221667      1714       5.7      -0.7        53     NaN

################### PROCESSING Sant Martí de Canals ###################

        Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Sant Martí de Canals 42.225278   0.990278       650       7.4       3.6         6     NaN

################### PROCESSING Solsona ##############

           Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
El Vendrell - Televisió 41.225556   1.538056        57      18.3      11.7        31     0.0

################### PROCESSING Els Pallaresos ###################

  Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Els Pallaresos 41.175556   1.269167       125      19.4      12.7        45     0.0

################### PROCESSING Falset ###################

Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
      Falset 41.146667   0.818611       367      21.4       4.1         0     0.4

################### PROCESSING Falset - Baboix ###################

   Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Falset - Baboix 41.140278   0.825556       370      21.9       4.2         0     0.6

################### PROCESSING Falset - Els Masos ###################

      Station Name  Latitude  Longitude  Al

             Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Ulldecona-S. Joan del Pas 40.607778   0.380833       205      25.0       8.7        45     NaN

################### PROCESSING Vallfogona de Riucorb ###################

         Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Vallfogona de Riucorb 41.564167   1.237778       575      19.4       6.9        27     NaN

################### PROCESSING Vallmoll ###################

Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
    Vallmoll 41.258611     1.2475       205      20.4       9.6        19     0.0

################### PROCESSING Valls - IES Narcís Oller ###################

            Station Name  Latitude  Longitude  Altitude  Temp_max  Temp_min  Wind_max  Precip
Valls - IES Narcís Oller 41.308056   1.266667       225      19.9       9.2        23     NaN

